# 1 class

In [1]:
from helpers.init import initialization
cfg, classes, labels_dict, train_dataset_combined, test_dataset_combined, valid_dataset_combined = initialization(merge_to_one_class = True, encoding = True, swin = False)


cfg.SOLVER.IMS_PER_BATCH = 2 # batch size
cfg.DATALOADER.NUM_WORKERS = 2
cfg.DATALOADER.PREFETCH_FACTOR = 2

 100% |█████████████████| 233/233 [762.3ms elapsed, 0s remaining, 305.6 samples/s]      
 100% |███████████████████| 51/51 [193.2ms elapsed, 0s remaining, 267.3 samples/s] 
 100% |███████████████████| 50/50 [193.6ms elapsed, 0s remaining, 260.3 samples/s] 
Found train dataset labels:
['sitting', 'walking', 'waving']
Found test dataset labels:
['sitting', 'walking', 'waving']
Found valid dataset labels:
['sitting', 'walking', 'waving']
 100% |███████████████████| 74/74 [210.4ms elapsed, 0s remaining, 351.7 samples/s]     
 100% |███████████████████| 17/17 [124.7ms elapsed, 0s remaining, 136.4 samples/s]    
 100% |███████████████████| 17/17 [129.0ms elapsed, 0s remaining, 131.8 samples/s]    
Found difficult_train dataset labels:
['sitting', 'walking', 'waving']
Found difficult_test dataset labels:
['sitting', 'walking', 'waving']
Found difficult_valid dataset labels:
['sitting', 'walking', 'waving']
Label dictionary for complete dataset
{'person': 0}


In [2]:
import os, cv2
from helpers.model_trainer import Predictor
from helpers.fiftyone_detectron2_bridge import clean_instances, detectron_to_fo, get_fiftyone_dicts

path = "output/0"
cfg.OUTPUT_DIR = path

for root, dirs, files in os.walk(path):
    for name in files:
        if '3_classes' not in root and 'inference' not in root and 'example_detector' not in root:
            if name[-3:] == 'pth':
                #print(os.path.join(root, name))

                print(f'Current model is:    {os.path.join(root, name)}')
                print('')
        
                weight_path = os.path.join(os.path.join(root, name))
                cfg.MODEL.WEIGHTS =  weight_path
                
                #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
                #cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.8
                
                predictor = Predictor(cfg)

                test_view = test_dataset_combined
                dataset_dicts = get_fiftyone_dicts(test_view, labels_dict)
                predictions = {}
                for d in dataset_dicts:
                    img_w = d["width"]
                    img_h = d["height"]
                    img = cv2.imread(d["file_name"], cv2.IMREAD_UNCHANGED)
                    outputs = predictor(img)
                    detections, instances = detectron_to_fo(outputs, img_w, img_h, classes, score_tresh=0.3)
                    predictions[d["image_id"]] = detections
                
                test_dataset_combined.set_values("predictions", predictions, key_field="id")

                test_results = test_dataset_combined.evaluate_detections(
                    "predictions",
                    gt_field="segmentations",
                    eval_key="eval",
                    use_masks=True,
                    compute_mAP=True,
                    classes=test_dataset_combined.distinct("segmentations.detections.label")
                )
                test_results.print_report()

Current model is:    output/0/model_0003999.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_in

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_16

Evaluating detections...
  75% |██████████████|----| 51/68 [424.4ms elapsed, 141.5ms remaining, 120.2 samples/s] 

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [505.3ms elapsed, 0s remaining, 134.6 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [457.3ms elapsed, 0s remaining, 148.7 samples/s]      
              precision    recall  f1-score   support

      person       0.71      0.43      0.54        95

   micro avg       0.71      0.43      0.54        95
   macro avg       0.71      0.43      0.54        95
weighted avg       0.71      0.43      0.54        95

Current model is:    output/0/model_0002499.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 

...

 100% |███████████████████| 68/68 [496.2ms elapsed, 0s remaining, 137.0 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [470.6ms elapsed, 0s remaining, 144.5 samples/s]      
              precision    recall  f1-score   support

      person       0.71      0.43      0.54        95

   micro avg       0.71      0.43      0.54        95
   macro avg       0.71      0.43      0.54        95
weighted avg       0.71      0.43      0.54        95

Current model is:    output/0/model_0001999.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [563.7ms elapsed, 0s remaining, 120.6 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [537.4ms elapsed, 0s remaining, 126.5 samples/s]      
              precision    recall  f1-score   support

      person       0.68      0.47      0.56       100

   micro avg       0.68      0.47      0.56       100
   macro avg       0.68      0.47      0.56       100
weighted avg       0.68      0.47      0.56       100

Current model is:    output/0/model_0001499.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [690.0ms elapsed, 0s remaining, 98.5 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [636.4ms elapsed, 0s remaining, 106.9 samples/s]      
              precision    recall  f1-score   support

      person       0.51      0.56      0.54       101

   micro avg       0.51      0.56      0.54       101
   macro avg       0.51      0.56      0.54       101
weighted avg       0.51      0.56      0.54       101



# 3 classes

In [1]:
from helpers.dataset import initialization
cfg, classes, labels_dict, train_dataset_combined, test_dataset_combined, valid_dataset_combined = initialization(merge_to_one_class = False, channels = [1, 2, 3])


cfg.SOLVER.IMS_PER_BATCH = 2 # batch size
cfg.DATALOADER.NUM_WORKERS = 2
cfg.DATALOADER.PREFETCH_FACTOR = 2

 100% |█████████████████| 233/233 [425.8ms elapsed, 0s remaining, 550.2 samples/s]      
 100% |███████████████████| 51/51 [129.8ms elapsed, 0s remaining, 399.6 samples/s] 
 100% |███████████████████| 50/50 [124.2ms elapsed, 0s remaining, 408.9 samples/s] 
Found train dataset labels:
['sitting', 'walking', 'waving']
Found test dataset labels:
['sitting', 'walking', 'waving']
Found valid dataset labels:
['sitting', 'walking', 'waving']
 100% |███████████████████| 74/74 [156.3ms elapsed, 0s remaining, 473.3 samples/s]     
 100% |███████████████████| 17/17 [53.8ms elapsed, 0s remaining, 315.8 samples/s]     
 100% |███████████████████| 17/17 [53.1ms elapsed, 0s remaining, 320.4 samples/s]     
Found difficult_train dataset labels:
['sitting', 'walking', 'waving']
Found difficult_test dataset labels:
['sitting', 'walking', 'waving']
Found difficult_valid dataset labels:
['sitting', 'walking', 'waving']
Means of the dataset:  [ 58.33375975 115.78446252 234.51154039]
STDs of the dataset:  [

In [2]:
import os, cv2
from helpers.model_trainer import Predictor
from helpers.fiftyone_detectron2_bridge import clean_instances, detectron_to_fo, get_fiftyone_dicts

path = "output/20"
cfg.OUTPUT_DIR = path

for root, dirs, files in os.walk(path):
    for name in files:
        if '1_class' not in root and 'inference' not in root and 'example_detector' not in root:
            if name[-3:] == 'pth':
                #print(os.path.join(root, name))

                print(f'Current model is:    {os.path.join(root, name)}')
                print('')
        
                weight_path = os.path.join(os.path.join(root, name))
                cfg.MODEL.WEIGHTS =  weight_path
                
                #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
                #cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.8
                
                predictor = Predictor(cfg)
        
                test_view = test_dataset_combined
                dataset_dicts = get_fiftyone_dicts(test_view, labels_dict)
                predictions = {}
                for d in dataset_dicts:
                    img_w = d["width"]
                    img_h = d["height"]
                    img = cv2.imread(d["file_name"], cv2.IMREAD_UNCHANGED)
                    outputs = predictor(img)
                    detections, instances = detectron_to_fo(outputs, img_w, img_h, classes, score_tresh=0.3)
                    predictions[d["image_id"]] = detections
                
                test_dataset_combined.set_values("predictions", predictions, key_field="id")

                test_results = test_dataset_combined.evaluate_detections(
                    "predictions",
                    gt_field="segmentations",
                    eval_key="eval",
                    use_masks=True,
                    compute_mAP=True,
                    classes=test_dataset_combined.distinct("segmentations.detections.label")
                )
                test_results.print_report()
                

Current model is:    output/20/model_0004499.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_i

/home/artemoneto/miniconda3/envs/maskdino/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Evaluating detections...
  72% |█████████████|-----| 49/68 [420.1ms elapsed, 162.9ms remaining, 116.6 samples/s] 

/home/artemoneto/miniconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [542.6ms elapsed, 0s remaining, 125.3 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [447.9ms elapsed, 0s remaining, 151.8 samples/s]      
              precision    recall  f1-score   support

     sitting       1.00      0.90      0.95        40
     walking       0.82      0.82      0.82        28
      waving       0.85      0.85      0.85        26

   micro avg       0.90      0.86      0.88        94
   macro avg       0.89      0.86      0.87        94
weighted avg       0.90      0.86      0.88        94

Current model is:    output/20/model_0001999.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'lo

...

 100% |███████████████████| 68/68 [535.8ms elapsed, 0s remaining, 126.9 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [467.4ms elapsed, 0s remaining, 145.5 samples/s]      
              precision    recall  f1-score   support

     sitting       1.00      0.90      0.95        40
     walking       0.79      0.70      0.75        27
      waving       0.85      0.88      0.87        26

   micro avg       0.90      0.84      0.87        93
   macro avg       0.88      0.83      0.85        93
weighted avg       0.90      0.84      0.87        93

Current model is:    output/20/model_0003999.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'lo

/home/artemoneto/miniconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [537.4ms elapsed, 0s remaining, 126.5 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [442.3ms elapsed, 0s remaining, 153.7 samples/s]      
              precision    recall  f1-score   support

     sitting       0.97      0.85      0.91        40
     walking       0.69      0.81      0.75        27
      waving       0.81      0.81      0.81        26

   micro avg       0.83      0.83      0.83        93
   macro avg       0.82      0.82      0.82        93
weighted avg       0.84      0.83      0.83        93

Current model is:    output/20/model_0002499.pth

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'lo

/home/artemoneto/miniconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [459.5ms elapsed, 0s remaining, 148.0 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [389.8ms elapsed, 0s remaining, 174.4 samples/s]      
              precision    recall  f1-score   support

     sitting       0.97      0.70      0.81        40
     walking       0.84      0.59      0.70        27
      waving       0.73      0.81      0.77        27

   micro avg       0.85      0.70      0.77        94
   macro avg       0.85      0.70      0.76        94
weighted avg       0.86      0.70      0.77        94



In [28]:
samples = 0
dets = 0

for sample in test_view:
    
    samples += 1
    print("\nSample number: ", samples)
    detections = sample.detections

    for detection in detections.detections:
        dets += 1
        print("Detection number: ", dets)
        #print(detection)


Sample number:  1
Detection number:  1

Sample number:  2
Detection number:  2

Sample number:  3
Detection number:  3

Sample number:  4
Detection number:  4

Sample number:  5
Detection number:  5

Sample number:  6
Detection number:  6

Sample number:  7
Detection number:  7

Sample number:  8
Detection number:  8

Sample number:  9
Detection number:  9

Sample number:  10
Detection number:  10

Sample number:  11
Detection number:  11

Sample number:  12
Detection number:  12

Sample number:  13
Detection number:  13

Sample number:  14
Detection number:  14

Sample number:  15
Detection number:  15

Sample number:  16
Detection number:  16

Sample number:  17
Detection number:  17

Sample number:  18
Detection number:  18

Sample number:  19
Detection number:  19

Sample number:  20
Detection number:  20

Sample number:  21
Detection number:  21

Sample number:  22
Detection number:  22

Sample number:  23
Detection number:  23

Sample number:  24
Detection number:  24

Sample nu